In [2]:
import os
import pandas as pd
import boto3
from datetime import datetime, timedelta

import importlib
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, FloatSlider
import order_book.transformation
from order_book.simulation import *
from order_book.transformation import *
from order_book.loader import *
import awswrangler as wr

from tensorflow.keras.callbacks import Callback
import ml_model
from ml_model.test_model import *

from tensorflow.keras.optimizers import Adam
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)

2024-05-27 10:50:21.388723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dl = DataLoader()

In [4]:
start = datetime(2024, 1, 1, 20)
end = datetime(2024, 1, 1, 21)

order_data = dl.get_raw_product_orders(start, end)

ClientError: An error occurred (UnrecognizedClientException) when calling the DeleteTable operation: The security token included in the request is invalid